In [1]:
import os
import sys
print(os.getcwd())
os.chdir('..')
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

/home/admin/coref_research/gpt-coref-met-1/src/analysis


In [2]:
import pickle
import torch
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from math import log2
from collections import Counter
import pandas as pd
from collections import defaultdict
import jsonlines
import os
from os import path
from torch import nn
import scipy
import copy
from utils.get_processed_dataset import *
from utils.utils import *
from config import *
import dtale
import json
from omegaconf import OmegaConf
from copy import deepcopy
from analysis.analysis_utils import *

In [3]:
os.getcwd()

'/home/admin/coref_research/gpt-coref-met-1/src'

In [4]:
def load_config(config_arg,model_name,run):
    dataset_config = OmegaConf.load(dataset_yaml)
    experiment_config = OmegaConf.load(config_arg)
    experiment_config["model"] = model_name
    experiment_config["run"] = run
    return dataset_config, experiment_config

In [5]:
def get_tbound_equivalent(sys_output, doc_processed):
    to_be_modified = ["predicted_clusters", "golden_clusters"]
    new_keys = ["predicted_clusters_stbound", "golden_clusters_stbound"]
    for doc_key in sys_output:
        st_map = doc_processed[doc_key]["subtoken_vs_token"]
        sys_output[doc_key]["predicted_clusters_stbound"] = deepcopy(sys_output[doc_key]["predicted_clusters"])
        sys_output[doc_key]["golden_clusters_stbound"] = deepcopy(sys_output[doc_key]["golden_clusters"])
        for to_modify in to_be_modified:
            for cluster in sys_output[doc_key][to_modify]:
                for mention in cluster:
                    print("Doc key: ", doc_key)
                    print("Before: ", mention)
                    print("Length of st_map: ", len(st_map))
                    mention[0] = st_map[mention[0]]
                    mention[1] = st_map[mention[1]]

In [6]:
def get_nested_mentions(mentions):
    mask_nested = [0 for _ in mentions]
    end_init = mentions[0][1]
    end_init_index = 0
    for mention_ind,mention in enumerate(mentions[1:],start=1):
        start,end = mention
        if start <= end_init:
            ## nested mention
            mask_nested[end_init_index] = 1
            mask_nested[mention_ind] = 1
            if end > end_init:
                end_init = end
                end_init_index = mention_ind 
        else:
            end_init = end
            end_init_index = mention_ind
    return mask_nested

In [7]:
def add_keys_to_output(sys_output):
    for document in sys_output:
        sys_output[document]["predicted_mentions"] = sorted([mention for cluster in sys_output[document]["predicted_clusters"] for mention in cluster])
        sys_output[document]["predicted_mentions_stbound"] = sorted([mention for cluster in sys_output[document]["predicted_clusters_stbound"] for mention in cluster])
        sys_output[document]["golden_mentions_stbound"] = sorted([mention for cluster in sys_output[document]["golden_clusters_stbound"] for mention in cluster])
        sys_output[document]["predicted_mentions_vs_clusters"] = {tuple(mention): cluster_ind for cluster_ind,cluster in enumerate(sys_output[document]["predicted_clusters"]) for mention in cluster}
        sys_output[document]["predicted_mentions_nested"] = get_nested_mentions(sys_output[document]["predicted_mentions"])
        sys_output[document]["gt_mentions"] = sorted([mention for cluster in sys_output[document]["golden_clusters"] for mention in cluster])
        sys_output[document]["gt_mentions_missing"] = [0 if mention not in sys_output[document]["predicted_mentions"] else 1 for mention in sys_output[document]["gt_mentions"]]
        sys_output[document]["gt_mentions_vs_clusters"] = {tuple(mention): cluster_ind for cluster_ind,cluster in enumerate(sys_output[document]["golden_clusters"]) for mention in cluster}
        sys_output[document]["gt_mentions_nested"] = get_nested_mentions(sys_output[document]["gt_mentions"])
    return sys_output

In [8]:
config_arg = "conf/final_table_configs/h2s/fantasy.yaml"
# config_arg = "../baseline_met/configs_met/final/lf/setup4/fantasy.yaml"
# config_arg = "../baseline_fc/configs/final/lf/setup4/litbank.yaml"
model_name = "gpt"
run = "run1"
dataset_config, experiment_config = load_config(config_arg,model_name,run)

doc_dataset = experiment_config["dataset"]
doc_split = experiment_config["split"]
doc_addr = dataset_config[doc_dataset][f"{doc_split}_file"]
doc_tsv_addr = dataset_config[doc_dataset]["tsv"]
is_sota = experiment_config.get("is_sota", False) 
sota_str = "_sota" if is_sota else ""
doc_me = dataset_config[doc_dataset][f"{doc_split}_me{sota_str}"]

head = experiment_config.get("head", False)

## Get the docs and train docs
print("Doc addr: ", doc_addr)
docs = get_coref_docs(doc_addr)
docs_processed = get_processed_dataset(docs,doc_tsv_addr,head=head)
major_entities = get_major_entities(doc_me)

litbank_config= {
                    "name": "LitBank",
                    "cluster_threshold": 1,
                    "canonical_cluster_threshold": 1,
                    "cross_val_split": 0,
                    "metrics":['MUC', 'Bcub', 'CEAFE']
                }

jsonl_file = experiment_config["paths"]["result_destination"]
print("Results jsonlines: ",jsonl_file)
file_name = os.path.splitext(jsonl_file)[0]
xlsx_file = f"{file_name}.xlsx"
txt_file = f"{file_name}_avg.txt"

sys_output = {}
with jsonlines.open(jsonl_file, mode='r') as reader:
    for line in reader:
        if line.get("golden_clusters", None) is None:
            line["golden_clusters"] = line["clusters"]
        sys_output[line['doc_key']] = line


Doc addr:  ../data/raw_data/fantasy/longformer_speaker/test.4096.jsonlines


/home/admin/miniconda3/envs/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



148_The_Lord’s_Animals_and_the_Devil’s_0
111_The_Skilful_Huntsman_0
198_Maid_Maleen_0
059_Frederick_and_Catherine_0
144_The_Donkey_0
074_The_Fox_and_His_Cousin_0
031_The_Girl_Without_Hands_0
130_One-eye,_Two-eyes,_and_Three-eyes_0
045_Thumbling_as_Journeyman_0
138_Knoist_and_his_Three_Sons_0
001_The_Frog-King,_or_Iron_Henry_0
008_The_Wonderful_Musician_0
109_The_Shroud_0
042_The_Godfather_0
039_The_Elves_0
156_Odds_And_Ends_0
136_Iron_John_0
114_The_Cunning_Little_Tailor_0
145_The_Ungrateful_Son_0
191_The_Sea-Hare_0
Results jsonlines:  ../results/final1/gpt/h2s/fantasy/test/full/desc/result.jsonl


In [9]:
if head == False:
    print("Reached here")
    get_tbound_equivalent(sys_output, docs_processed)
    original_index_list_key = "mentions_vs_tbound"
else:
    original_index_list_key = "mentions_vs_headbound"

sys_output = add_keys_to_output(sys_output)

Reached here
Doc key:  059_Frederick_and_Catherine_0
Before:  [16, 16]
Length of st_map:  3369
Doc key:  059_Frederick_and_Catherine_0
Before:  [45, 45]
Length of st_map:  3369
Doc key:  059_Frederick_and_Catherine_0
Before:  [81, 81]
Length of st_map:  3369
Doc key:  059_Frederick_and_Catherine_0
Before:  [87, 87]
Length of st_map:  3369
Doc key:  059_Frederick_and_Catherine_0
Before:  [103, 103]
Length of st_map:  3369
Doc key:  059_Frederick_and_Catherine_0
Before:  [144, 144]
Length of st_map:  3369
Doc key:  059_Frederick_and_Catherine_0
Before:  [158, 158]
Length of st_map:  3369
Doc key:  059_Frederick_and_Catherine_0
Before:  [162, 162]
Length of st_map:  3369
Doc key:  059_Frederick_and_Catherine_0
Before:  [171, 171]
Length of st_map:  3369
Doc key:  059_Frederick_and_Catherine_0
Before:  [180, 180]
Length of st_map:  3369
Doc key:  059_Frederick_and_Catherine_0
Before:  [193, 193]
Length of st_map:  3369
Doc key:  059_Frederick_and_Catherine_0
Before:  [225, 225]
Length of s